In [7]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
import pandas as pd
!pip install seaborn
 
# For visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [9]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

table_name = 'PHARMASALES'

sf_df = my_session.sql("select * from {}".format(table_name))

df = sf_df.to_pandas()
 

SnowparkSQLException: (1304): 01b8788c-0004-323d-0000-31ef003ce026: 002003 (42S02): SQL compilation error:
Object 'PHARMASALES' does not exist or not authorized.